# **Exploring the Steane Code in pyQuEST**

---


The Steane code is a famous quantum error-correcting code that encodes a single logical qubit in seven physical qubits. Also known as a **[[7, 1, 3]]** CSS code, the Steane code is the smallest qubit CSS code capable of correcting a single error, whether a phase-flip or bit-flip.

Quantum error correction is essential for protecting quantum information from errors due to decoherence, imperfect gate operations, and other noise sources. The Steane code is a simple example of how such errors might be handled in quantum systems, and using pyQuEST, we can probe this quantitatively. **In particular, we can use pyQuEST to determine the rate of logical errors, or how often the logical encoding will change states.** <br><br>

#### **Objectives**

To delve deeper into the Steane code's capabilities and practical implementations, we will achieve the following objectives:

1. **Review the composition of the Steane code and understand its error-correcting capabilities.**

2. **Construct the Steane code using pyQuEST for a Monte Carlo simulation.**

3. **Construct the Steane code using pyQuEST for a density matrix simulation.**

In our pyQuEST simulations, we will investigate the performance of the code for storing a logical state in the presence of noise.<br><br>


## **Composition of the Steane Code**

At the core of the Steane code (and indeed quantum error-correction codes more generally) are a set of **stabilisers**. In the Steane code, the sets of $X$ and $Z$ stabilisers is given by:<br><br>

$S_X = \{X_0 X_1 X_2 X_3, X_1 X_2 X_4 X_5, X_2 X_3 X_5 X_6 \}$

$S_Z = \{Z_0 Z_1 Z_2 Z_3, Z_1 Z_2 Z_4 Z_5, Z_2 Z_3 Z_5 Z_6 \}$ <br><br>

The Steane code can be nicely represented with a triangle, with each coloured face (or plaquette) supporting an $X$ and a $Z$ stabiliser:

<div style="text-align: center;">
    <img src="images/steane_triangle.jpg" alt=" of Steane Code Triangle" width="500" height="500">
</div>

We can see how the stabilisers above can be matched to the triangle &mdash; each of the pairs of $X$ and $Z$ stabilisers can be placed on a distinct plaquette. Using pyQuEST, we can build these stabilisers.



### **Stabilisers in pyQuEST**

Before we begin, we need to define a qubit register &mdash; in the case of the Steane code, we have seven data qubits. We also need an ancilla that we will recycle throughout our code.

In [ ]:
# Import the libraries for constructing a register and a circuit with relevant gates
from pyquest import Circuit, Register
from pyquest.unitaries import H, X


# Define an eight-qubit register - seven data qubits and one ancilla
reg = Register(8)

Let's now implement the $X$ stabilisers &mdash; these will help us to correct phase-flip errors. We can define a general circuit that contains the operators we need:

In [ ]:
# Set the ancilla qubit
ancilla = 7

# Set the stabiliser qubits - e.g. for the red plaquette
qubits = [0, 1, 2, 3]

# Define a circuit with the operators for an X stabiliser
x_stabiliser_circ = Circuit([
    H(ancilla), X(qubits[0], controls=ancilla), X(qubits[1], controls=ancilla), 
    X(qubits[2], controls=ancilla), X(qubits[3], controls=ancilla), H(ancilla)
])

We can similarly implement the $Z$ stabilisers &mdash; these will help us to correct bit-flip errors:

In [ ]:
# Define a circuit with the operators for a Z stabiliser
z_stabiliser_circ = Circuit([
    X(ancilla, controls=qubits[0]), X(ancilla, controls=qubits[1]), 
    X(ancilla, controls=qubits[2]), X(ancilla, controls=qubits[3]),
])

After each stabiliser circuit is run, we must include measurement operators that allow us to identify errors. 

>Note that in pyQuEST, the results of a particular measurement operator is returned as an array &mdash; this makes it easy to access the results of an operator that is applied over several qubits. If the measurement is applied as part of a circuit, results are returned as a 2D array, with each 1D array containing the results from a measurement operator in the circuit, in order. Since we plan to include the measurement operator as part of the circuit and want to measure only the ancilla qubit, the result of our measurement must be accessed using 2D indexing.

Given that we are only using one ancilla, we must also include a 'correction' that resets the ancilla if the measurement result is -1 (indicating a 1 state). Putting everything together in functions:

In [ ]:
# Import the measurement operator
from pyquest.gates import M


# Define a function for implementing the X stabiliser
def x_stabiliser(ancilla, qubits):
    circ = Circuit([
        H(ancilla), X(qubits[0], controls=ancilla), X(qubits[1], controls=ancilla), 
        X(qubits[2], controls=ancilla), X(qubits[3], controls=ancilla), H(ancilla),
        M([ancilla])
    ])

    # Apply the circuit - the output will contain the state of the ancilla
    measurement = reg.apply_circuit(circ)

    # Access the measurement result using 2D indexing
    # If the ancilla is in a 1 state, reset the ancilla and return -1
    if measurement[0][0] == 1:
        reg.apply_operator(X(ancilla))
        return -1
    # Else if the ancilla is in a 0 state, just return +1
    else:
        return 1
        

def z_stabiliser(ancilla, qubits):
    circ = Circuit([
        X(ancilla, controls=qubits[0]), X(ancilla, controls=qubits[1]), 
        X(ancilla, controls=qubits[2]), X(ancilla, controls=qubits[3]),
        M([ancilla])
    ])

    measurement = reg.apply_circuit(circ)

    if measurement[0][0] == 1:
        reg.apply_operator(X(ancilla))
        return -1
    else:
        return 1

Now that we have the stabiliser framework, we can get to work on using this in a Monte Carlo simulation.<br><br>

## **Monte Carlo Implementation**

In a Monte Carlo approach, we aim to run a large number of trials in order to obtain useful statistical estimates of our code performance, particularly in the presence of noise. Before generating noise models, let's first explore how we can put together a simulation that can repeatedly runs our stabilisers. 

Above, we defined a single set of qubits corresponding to one of the plaquettes. We can deifne a 2D array containing all three qubit combinations:

In [ ]:
# Define the sets of qubits needed for the Steane code stabiliser measurements
chosen_qubits = [[0, 1, 2, 3], [1, 2, 4, 5], [2, 3, 5, 6]]

For our simulations, we will consider an encoded 0 logical state. We will need to prepare a perfect such state before any live runs &mdash; to do so, we will need a **decoder**. Not only will this be useful for preparing a perfect logical state, but the decoder will also be central to correcting real errors when we start to consider noise models.

When first running our $Z$ stabilisers, we should obtain a perfect syndrome &mdash; i.e. all +1 stabiliser results. When first running our $X$ stabilisers however, the syndrome may not be perfect &mdash; each stabiliser has a 50% chance of returning +1, and a 50% chance of returning -1. We will want to perform 'corrections' that will not affect our logical state but will allow us to start with a system that always returns +1 stabiliser results in the absence of noise.

We can build our own decoder using a lookup table, stored as a Python dictionary. If we consider all possible error syndromes for one stabiliser type (there are three results with two possibilities, so $2^3 = 8$ combinations) and the corresponding qubit error locations, we can devise the lookup table below:

In [ ]:
# Match the possible syndromes to the corresponding error locations
syndrome_lookup = {
    "[-1, 1, 1]" : 0,
    "[-1, -1, 1]" : 1,
    "[-1, -1, -1]" : 2,
    "[-1, 1, -1]" : 3,
    "[1, -1, 1]" : 4,
    "[1, -1, -1]" : 5,
    "[1, 1, -1]" : 6
}

Note that the order of the stabiliser results as seen in the syndrome is given by the order in **chosen_qubits** &mdash; in the Steane triangle, this is the red plaquette, then the blue plaquette, and then finally the green plaquette. **[1, 1, 1]** is not stored as this syndrome requires no corrections. 

To see how we obtained the above, choose a data qubit and consider the effect of an error on the stabiliser results: if there was an error on data qubit 4, only the blue plaquette (or stabiliser number two) would indicate an error, leading to a -1 result for that stabiliser. If there was an error on data qubit 2 instead, all three plaquettes would indicate an error, leading to -1 results for all three stablisers.

With this lookup table, we can define a function that takes a syndrome (as a string) and a correction operator, and performs the desired correction:

In [ ]:
def perform_correction(results, gate):
    # Check if the syndrome is in the lookup table
    if results in syndrome_lookup:
        # If it is in the lookup table, find the target qubit and apply the given correction operator
        target_qubit = syndrome_lookup[results]
        reg.apply_operator(gate(target_qubit))

With these pieces in place, not only can we prepare a perfect logical 0 state, but we have now also laid the groundwork for the stabiliser cycles that will be the core of our simulations.<br><br>

### **Putting it Together**

Our Monte Carlo simulation will involve preparing a perfect logical 0 state, and running stabiliser cycles with corrections a large number of times. We can run the code below before our live runs to prepare the state and within our live loop to perform our stabiliser cycles:

In [ ]:
# Import the Z operator
from pyquest.unitaries import Z


# Create arrays to store X and Z stabiliser results
x_stabiliser_results = []
z_stabiliser_results = []

# Perform all three X stabilisers, and all three Z stabilisers
# Store the measurement results in the arrays above
for i in range(3):
    x_stabiliser_results.append(x_stabiliser(7, chosen_qubits[i]))
for j in range(3):
    z_stabiliser_results.append(z_stabiliser(7, chosen_qubits[j]))
    
# Apply phase-flips given the X stabiliser results, and bit-flips given the Z stabiliser results 
perform_correction(str(x_stabiliser_results), Z)
perform_correction(str(z_stabiliser_results), X)

We will then need to construct a loop for our live runs. If, for example, we want to run 1,000,000 stabiliser cycles in our simulation, we would run:

In [ ]:
# Set the number of runs to 1,000,000
num_runs = 1000000

for run in range(num_runs):
    
    x_stabiliser_results = []
    z_stabiliser_results = []

    for i in range(3):
        x_stabiliser_results.append(x_stabiliser(7, chosen_qubits[i]))
    for j in range(3):
        z_stabiliser_results.append(z_stabiliser(7, chosen_qubits[j]))

    perform_correction(str(x_stabiliser_results), Z)
    perform_correction(str(z_stabiliser_results), X)

So far, our code has run only clean stabiliser cycles: every set of stabilisers returns perfect syndromes. This does not offer any real insights into the Steane code's performance.

By introducing noise, however, we will be able to observe how well the code does its job of preserving the correct logical state.<br><br>

### **Exploring Noise**

In real quantum computing systems, noise can be quite complex and involve several poorly understood degrees of freedom. For our purposes, we will only consider noise that is 'environmental,' acting on each data qubit independently and only once per cycle before all stabilisers are performed. 

>Note that we could also explore a wide range of other noise models, with pyQuEST offering a high degree of customisability.

Such a noise model can be implemented using a random number generator, where an error on a particular data qubit is only applied if the random number is less than a given error probability, for instance 0.1%:

In [ ]:
# Import the Y operator and Python's 'random' library
from pyquest.unitaries import Y
import random


# Store the possible error operators
error_gates = [X, Y, Z]

# Set the desired error probability
error_prob = 0.001

# Loop through every data qubit
for qubit in range(7):

    # Create an array for storing the chosen errors
    chosen_errors = []

    # If the random number is less than the error probability, choose an error and store it
    if random.random() < error_prob:
        chosen_errors.append((random.choice(error_gates))(qubit))

In order for a logical error to occur &mdash; where our encoded logical state flips from 0 to 1 &mdash; our system will need to face at least two errors at a time. This is because the Steane code can handle one error perfectly &mdash; only when there is a combination of errors disguised as a single error in the syndrome can a logical error occur. As a result, we can optimise our code so that the stabiliser cycle only runs when two or more errors occur. 

We also need to keep track of any logical errors that occur during our simulation &mdash; since performing a logical measurement will destroy quantum information, we do not want to perform any such measurements to the qubits in our simulation. Instead, in pyQuEST, we can make an exact copy of our register and perform destructive measurements on that copy, discarding it afterwards. In the Steane code, measuring along any edge of the triangle will give us the logical state of the qubit, so we can choose to measure qubits 0, 1, and 4, and check the parity of that measurement. **A logical error will be counted if the current logical state does not match the logical state of the previous stabiliser cycle.**

Putting our overall simulation loop together:

In [ ]:
# Create a variable to store the number of logical errors
num_log_errors = 0

# Create a variable to store the logical state of the previous cycle
# We will use this to check for logical flips in the current cycle
prev_logical = 0

for run in range(num_runs):

    chosen_errors = []

    for qubit in range(7):

        if random.random() < error_prob:
            chosen_errors.append((random.choice(error_gates))(qubit))

    # If the number of errors is less than 2, skip the run
    if len(chosen_errors) < 2:
            continue
    # Else store the errors and continue with the stabiliser cycle
    else:
        for error in chosen_errors:
            reg.apply_operator(error)
                

    x_stabiliser_results = []
    z_stabiliser_results = []

    for i in range(3):
        x_stabiliser_results.append(x_stabiliser(7, chosen_qubits[i]))
    for j in range(3):
        z_stabiliser_results.append(z_stabiliser(7, chosen_qubits[j]))

    perform_correction(str(x_stabiliser_results), Z)
    perform_correction(str(z_stabiliser_results), X)


    # Creates a copy of the register
    measure_reg = Register(copy_reg = reg)

    # Performs a destructive logical measurement
    # The parity of the states measured gives the logical state
    logical_result = sum(measure_reg.apply_operator(M([0, 1, 4]))) % 2

    # Checks if there has been a logical flip
    if prev_logical != logical_result:
        num_log_errors += 1
        # Stores the new logical state for comparison in the next cycle
        prev_logical = logical_result

Finally, if we wanted to find the logical error rate, we can just divide the number of logical errors by the number of live runs:

In [ ]:
logical_rate = num_log_errors/num_runs

print("Logical Error Rate:", logical_rate)

We have now constructed a Monte Carlo approach to simulating the performance of the Steane code &mdash; running this over a range of physical error rates (the error probability given to the simulation) will allow you to analyse how this performance varies as the error rate increases.<br><br>

## **Density Matrix Implementation**

In the density matrix approach, we need to:

- consider all possible outcomes in a stabiliser cycle;
- keep track of the probabilities of each outcome;
- and sum together the weighted density matrices for all of these outcomes.

This means a few important differences compared to the Monte Carlo approach.<br><br>

### **Setting Up**

We will again need to define a register in pyQuEST, this time with the **density_matrix** parameter set to True. As before, we will also need to define functions for our stabilisers &mdash; in this approach, however, we will not include measurement operators in our circuit constructions. Using pyQuEST's built-in features, we will later make non-destructive measurements of the probabilities defining a qubit's state.

Note that our stabiliser functions will also take **register** as an argument &mdash; this will be important for when we perform operations on copy density matrices as we consider all possible stabiliser outcomes.

In [ ]:
from pyquest import Circuit, Register
from pyquest.unitaries import H, X

# Define an eight-qubit register - make it a density_matrix
reg = Register(8, density_matrix = True)


# Define a function for implementing an X stabiliser (without measurement), taking register as an argument
def x_stabiliser(ancilla, qubits, register):
    circ = Circuit([
        H(ancilla), X(qubits[0], controls=ancilla), X(qubits[1], controls=ancilla), 
        X(qubits[2], controls=ancilla), X(qubits[3], controls=ancilla), H(ancilla)
    ])

    register.apply_circuit(circ)
        

def z_stabiliser(ancilla, qubits, register):
    circ = Circuit([
        X(ancilla, controls=qubits[0]), X(ancilla, controls=qubits[1]), 
        X(ancilla, controls=qubits[2]), X(ancilla, controls=qubits[3])
    ])
        
    register.apply_circuit(circ)

While the lookup table we will use is similar to the Monte Carlo approach, we will need to replace the measurement outcomes, +1 and -1, with the actual measured states, 0 and 1 respectively. This will allow us to work with stabiliser outcomes using binary representations.

Our correction function will remain the same, just taking the additional register argument as in the stabiliser functions above:

In [ ]:
# Match the possible stabiliser outcomes to the corresponding error locations
syndrome_lookup = {
    "[1, 0, 0]" : 0,
    "[1, 1, 0]" : 1,
    "[1, 1, 1]" : 2,
    "[1, 0, 1]" : 3,
    "[0, 1, 0]" : 4,
    "[0, 1, 1]" : 5,
    "[0, 0, 1]" : 6
}


def perform_correction(results, register, gate):
    # Check if the outcome set is in the lookup table
    if results in syndrome_lookup:
         # If it is in the lookup table, find the target qubit and apply the given correction operator
        target_qubit = syndrome_lookup[results]
        register.apply_operator(gate(target_qubit))

We can initialise a perfect logical 0 state in a similar way to our Monte Carlo approach:

In [ ]:
from pyquest.unitaries import Z


# Define the sets of qubits needed for the Steane code stabiliser measurements
chosen_qubits = [[0, 1, 2, 3], [1, 2, 4, 5], [2, 3, 5, 6]]

# Create an array to store the stabiliser measurements
measurement = []


# Perform all three X stabilisers, and then all three Z stabilisers
# Store the measurement results for each in the array above
for x_round in range(0,3):
    x_stabiliser(7, chosen_qubits[x_round], reg)
    measurement.append(reg.apply_operator(M([7]))[0])
    # Reset the ancilla if the most recent measurement was 1
    if measurement[x_round] == 1:
        reg.apply_operator(X(7))


for z_round in range(3,6):
    # Adjust the indexing for chosen_qubits to cycle through all sets again
    z_stabiliser(7, chosen_qubits[z_round - 3], reg)
    measurement.append(reg.apply_operator(M([7]))[0])
    if measurement[z_round] == 1:
        reg.apply_operator(X(7))


# Extract the outcomes from the X stabilisers, and then from the Z stabilisers
# Pass the outcome string into the correction function
perform_correction(str(measurement[0:3]), reg, Z)
perform_correction(str(measurement[3:6]), reg, X)

With our logical state prepared, we can now focus on setting up the core of our simulation.<br><br>

### **Constructing the Stabiliser Cycle Loop**

Unlike in the Monte Carlo approach, in the density matrix approach we only need to run our stabiliser cycle a small number of times in order to understand the performance of the code for a particular physical error rate. The slope when plotting the probability of measuring a logical 1 state against runs gives the logical error rate, so only a few runs &mdash; for instance, less than ten &mdash; provide sufficient information to understand the code's performance.

For every stabiliser cycle, we need to accumulate the density matrices of all possible stabiliser cycle outcomes. We therefore need to create a blank density matrix (where all the probability amplitudes are 0) that will become the weighted sum of all possible density matrix outcomes.

Just as in the Monte Carlo approach, we want to investigate the code under noise &mdash; we can use a similar environmental noise model, applying **depolarising** noise to each data qubit at the start of every stabiliser cycle.

In [ ]:
# Import the depolarising noise operator
from pyquest.decoherence import Depolarising


# Set the number of full stabiliser cycles - e.g. 5
num_runs = 5


for run in range(num_runs):

    # Apply depolarising noise to each data qubit
    for i in range(7):
        reg.apply_operator(Depolarising(i, prob=error_prob))
            
    # Create an accumulator to store the weighted sum of density matrices
    accumulator_reg = Register(8, density_matrix = True)
    accumulator_reg.init_blank_state()

Since there are six stabiliser measurements, each with two possible state measurements, there are $2^6 = 64$ different measurement combinations. We can consider all 64 using binary, taking the following steps for each combination: <br><br>

1. **Create a copy of the current or 'master' density matrix and perform subsequent operations on this.**
2. **For each bit in the binary representation in turn, apply the corresponding stabiliser ($X$ or $Z$), update a running total with the probability of measuring the state given by the value of the bit, and force the stabiliser into 'measuring' that outcome (thereby collapsing the state).**
3. **Use the lookup table to perform corrections &mdash; just as in a normal stabiliser cycle &mdash; based on the overall measurement outcome (equivalent to the current binary representation).**
4. **Add the copy register multiplied by the running total &mdash; representing the overall probability of seeing the particular measurement outcome &mdash; and add this to the accumulator.**<br><br>

>Note that pyQuEST allows an indirect (non-destructive) measurement of the probabilities of measuring 0 or 1 on a particular qubit with **register.prob_of_all_outcomes([qubits])**. The probability of 0 can be accessed at the 0th index, and the probability of 1 can be accessed at the 1st index.

<br><br>Once all 64 measurement combinations have been worked through, update the master density matrix with the accumulator. The master density matrix now contains the weighted sum of all possible measurement outcomes, and will be used as the basis for the next stabiliser cycle.

Putting these ideas together, the code within each **run** loop will look like:

In [ ]:
# Loop through every measurement combination
for k in range (64):


    # Initialise a running total that will store the overall measurement probability
    meas_prob = 1

    # Create a binary representation of the loop number, fixed at six bits (for the six stabiliser measurements)
    binary_rep = [int(bit) for bit in f'{k:06b}']  

    # Make a copy of the master density matrix
    # Use this for operations applied for the current measurement combination
    copy_reg = Register(copy_reg = reg)


    # Loop through the three X stabilisers
    for x_qubit in range(0,3):

        # Perform the X stabiliser
        x_stabiliser(7, chosen_qubits[x_qubit], copy_reg)
            
        # Find the probability of measuring the outcome specified by the corresponding bit in the binary number
        meas_prob *= (copy_reg.prob_of_all_outcomes([7])[binary_rep[x_qubit]])

        # Force a measurement of that specified outcome
        copy_reg.apply_operator(M([7], force=binary_rep[x_qubit]))

        # Reset the ancilla if the measurement was of state 1
        if binary_rep[x_qubit] == 1:
            copy_reg.apply_operator(X(7))


    for z_qubit in range(3,6):
        
        z_stabiliser(7, chosen_qubits[z_qubit - 3], copy_reg)

        meas_prob *= (copy_reg.prob_of_all_outcomes([7])[binary_rep[z_qubit]])

        copy_reg.apply_operator(M([7], force=binary_rep[z_qubit]))

        if binary_rep[z_qubit] == 1:
            copy_reg.apply_operator(X(7))


    # Perform corrections based on the stabiliser results
    perform_correction(str(binary_rep[0:3]), copy_reg, Z)
    perform_correction(str(binary_rep[3:6]), copy_reg, X)

    # Add the weighted density matrix to the accumulator
    accumulator_reg += meas_prob * copy_reg
                

# After all combinations, set the master density matrix to the accumulator
reg = accumulator_reg

After updating the register with the accumulator, we will also need to measure the probability of measuring a logical 1 state. We do not want to destructively measure this out &mdash; we can instead apply CNOTs that outputs the logical state onto the ancilla, make an indirect measurement of the state probabilities, and then reverse the circuit such that the system is left unchanged.

The measurement function will look like:

In [ ]:
def measure_logical(ancilla, qubits, register):

    circ = Circuit([
        X(ancilla, controls=qubits[0]),
        X(ancilla, controls=qubits[1]),
        X(ancilla, controls=qubits[2])
    ])

    # Apply the measurement circuit
    register.apply_circuit(circ)
    # Store the probability of measuring a 1
    prob = register.prob_of_all_outcomes([ancilla])
    # Apply the inverse of the measurement circuit to leave the system unchanged
    register.apply_circuit(circ.inverse) 

    return prob

All that is left to do is to add a logical check at the end of each stabiliser cycle, adding the probability of measuring a logical 1 to a storage array:

In [ ]:
# BEFORE THE OVERALL RUNS LOOP

# Create an array to store the probabilities of a logical 1 for every cycle
logical_prob = []


# ......


# INSIDE THE OVERALL RUNS LOOP, AT THE END

# Use a set of edge qubits, as in the Monte Carlo carlo approach
logical_prob.append(measure_logical(7, [0, 1, 4], reg)[1])
print("Probability of Logical 1:", logical_prob[run])

## **Conclusion**

There we have it! In pyQuEST, we have seen how to implement and analyse a fundamental quantum error-correcting code through both Monte Carlo and density matrix simulations. These methods give us valuable insights into the code's effectiveness at handling errors and preserving the integrity of logical qubits. 

See if you can implement other codes and explore their performance under various noise models!